In [23]:
from twython import Twython
import json
credentials = {}
credentials['CONSUMER_KEY'] = "pyKBZeHhRb9w006Cv4Ituqi7E"
credentials['CONSUMER_SECRET'] = "DwUyJXTzPF4FexqztrXKu6ij7tf9GHnH9EiGGh0FGtdgAejjux"
credentials['ACCESS_TOKEN'] = "53697152-Nvdb7vmwGzPUI65duv6fkkA5ejvC8JSt0FtZncz3J"
credentials['ACCESS_SECRET'] = "AlQ0boi3ePIY3muIH6DHylCidVyMkEtI56hI9cin2C7wx"




In [25]:
# Instantiate an object
python_tweets = Twython(credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'])

# Create our query
query = {'q': 'learn python',
        'result_type': 'popular',
        'count': 10,
        'lang': 'en',
        }

In [26]:
import pandas as pd

# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}
for status in python_tweets.search(**query)['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(5)

,user,date,text,favorite_count
1,ThePSF,Fri Aug 09 23:55:00 +0000 2019,The PSF is looking for bloggers! Interview pro...,112
3,anacondainc,Tue Aug 06 17:13:30 +0000 2019,Why traders and finance professionals need to ...,93
4,KirkDBorne,Wed Aug 07 15:43:43 +0000 2019,Learn the good stuff about #AI: https://t.co/c...,69
0,KirkDBorne,Sat Aug 10 23:44:10 +0000 2019,[FREE e-Book] Learn Classification and Regress...,48
2,KirkDBorne,Sun Aug 11 22:51:00 +0000 2019,The best #MachineLearning and #DeepLearning Li...,18


In [59]:
# Create our query
query = {'q': 'learn python',
        'result_type': 'popular',
        'count': 10,
        'lang': 'en',
        }
tweetsarr = []
def process_tweet(tweet):
    d = {}
    #d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['location'] = tweet['user']['location']
    return d

for status in python_tweets.search(**query)['statuses']:
    
    #tweets.append(status)
    print(process_tweet(status))    
    tweetsarr.append(process_tweet(status))

tweets = pd.DataFrame(data=tweetsarr)
tweets.head()

{'text': '[FREE e-Book] Learn Classification and Regression in a Weekend: https://t.co/I4CTsFuNZG\n—————\n#BigData… https://t.co/MprKCQALdz', 'user': 'KirkDBorne', 'location': 'Booz Allen Hamilton'}
{'text': 'The PSF is looking for bloggers! Interview prominent Pythonistas, connect with the community, expand your circle of… https://t.co/QiDa8o56Gu', 'user': 'ThePSF', 'location': 'Everywhere Python is!'}
{'text': 'The best #MachineLearning and #DeepLearning Libraries — #TensorFlow, Spark MLlib, Scikit-learn, #PyTorch, MXNet, an… https://t.co/FpkCsQCdO2', 'user': 'KirkDBorne', 'location': 'Booz Allen Hamilton'}
{'text': 'Why traders and finance professionals need to learn Python by @SemikaSemika https://t.co/2Ic5nwmGon', 'user': 'anacondainc', 'location': 'Austin, TX'}
{'text': 'Learn the good stuff about #AI: https://t.co/c2emzJviud\n—————\n#BigData #DataScience #Algorithms #NeuralNetworks… https://t.co/n9nLczi52X', 'user': 'KirkDBorne', 'location': 'Booz Allen Hamilton'}


,text,user,location
0,[FREE e-Book] Learn Classification and Regress...,KirkDBorne,Booz Allen Hamilton
1,The PSF is looking for bloggers! Interview pro...,ThePSF,Everywhere Python is!
2,The best #MachineLearning and #DeepLearning Li...,KirkDBorne,Booz Allen Hamilton
3,Why traders and finance professionals need to ...,anacondainc,"Austin, TX"
4,Learn the good stuff about #AI: https://t.co/c...,KirkDBorne,Booz Allen Hamilton


In [41]:
from twython import TwythonStreamer
import csv

# Filter out unwanted data
def process_tweet(tweet):
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d
    
    
# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):
        print("----------------")
        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()
        
    # Save each tweet to csv file
    def save_to_csv(self, tweet):
        with open(r'saved_tweets.csv', 'a',encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))

In [35]:
# Instantiate from our streaming class
stream = MyStreamer(credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'], 
                    credentials['ACCESS_TOKEN'], credentials['ACCESS_SECRET'])
# Start the stream
stream.statuses.filter(track='python')

----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------


KeyboardInterrupt: 

In [43]:
from collections import Counter
import ast

tweets = pd.read_csv("saved_tweets.csv", encoding='ISO-8859-1')



In [74]:
from geopy.geocoders import Nominatim
import gmplot

geolocator = Nominatim()

# Go through all tweets and add locations to 'coordinates' dictionary
coordinates = {'latitude': [], 'longitude': []}
for count, user_loc in enumerate(tweets.location):
    try:
        location = geolocator.geocode(user_loc)
        
        # If coordinates are found for location
        if location:
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)
            
    # If too many connection requests
    except:
        pass



# Instantiate and center a GoogleMapPlotter object to show our map
gmap = gmplot.GoogleMapPlotter(30, 0, 3)

# Insert points on the map passing a list of latitudes and longitudes
gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20,
             gradient =((200, 200, 200, 0.6),(100, 100, 100, 0.3),(50, 50, 50, 0.3)))

gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20,
             gradient =((100, 100, 100, 0.6),(200, 200, 200, 0.3),(50, 50, 50, 0.3)))

# Save the map to html file
gmap.draw("python_heatmap.html")

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.
